In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

### Load and push a document to  the DB

In [ ]:
loader = TextLoader('document.txt', encoding='utf-8')
documents = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunck_size=1000, chunk_overlap=80)
texts = text_splitter.split_documents(documents)
print(len(texts))

In [ ]:
embeddings = OpenAIEmbeddings()
vector = embeddings.embed_query('Testing the embedding model')
doc_vectors = embeddings.embed_documents([t.page_content for t in texts])

In [ ]:
from langchain.vectorstores.pgvector import PGVector

db = PGVector.from_documents(
    embedding=embeddings,
    documents=texts,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING
)

### Retrieve a document from the DB

In [ ]:
retriever = db.as_retriever(search_kwargs={'k':3})

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    chain_type='stuff'
)

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template="""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
)

In [ ]:
chain_type_kwargs = {'prompt': PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=retriever,
    chain_type='stuff',
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
query = ""

qa.run(query)